# Imports

In [145]:
import pickle
import numpy as np
import geopandas as gpd
from mgwr.gwr import GWR
from mgwr.sel_bw import Sel_BW
import libpysal.weights as weights
import pysal.explore as esda
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [146]:
np.random.seed(42)

# Load data

In [147]:
df = gpd.read_file("datasets/5_split/df_fe.gpkg")

# Separate features

In [148]:
features = df.copy()

# Dependent variables
labels = features.pop('very_good_health')

# Outer CV folds
outer_fold_ids = df["outer_loop_fold_id_python"]
outer_splits = np.sort(outer_fold_ids.unique().astype(int))

# Inner CV folds
inner_fold_ids = df[[col for col in df.columns if "fold_id_python" in col]]
inner_splits = np.sort(inner_fold_ids.stack().unique().astype(int))

# Independent variables
features = df.drop(columns = [col for col in df.columns if "fold_id" in col])

# Functions

In [149]:
def get_random_hyperparameters():
    kernel = np.random.choice(["bisquare", "Gaussian", "exponential"])
    criterion = np.random.choice(["AICc", "AIC", "BIC", "CV"])
    return kernel, criterion

## Get GWR inputs

In [150]:
def get_gwr_inputs(features, labels, bandwidth = False, kernel = None, criterion = None):
    x = features.geometry.x
    y = features.geometry.y
    coords = np.array(list(zip(x, y)))
    target = labels.values.reshape((-1, 1))
    predictor_cols = ["greenspace_proportion", "imd", "f_m_ratio", "mean_age"]
    predictors = np.hstack(
        [features[col].values.reshape((-1, 1)) for col in predictor_cols]
    )
    opt_bandwidth = None
    if bandwidth:
        opt_bandwidth = Sel_BW(coords, target, predictors, kernel = kernel).search(criterion = criterion)
    return coords, predictors, target, opt_bandwidth


In [ ]:
def get_evaluation_metrics(val_features, val_labels, predictions):
    mae = mean_absolute_error(val_labels, predictions)
    mse = mean_squared_error(val_labels, predictions)
    r2 = r2_score(val_labels, predictions)
    w = weights.KNN.from_dataframe(val_features, k = 8)
    moran = esda.esda.Moran(val_labels.values - predictions.flatten(), w)
    return mae, mse, r2, moran

In [152]:
def get_avg_scores(cv_results):
    mae_scores = []
    mse_scores = []
    r2_scores = []

    for result in cv_results:
        mae_scores.append(result["mae"])
        mse_scores.append(result["mse"])
        r2_scores.append(result["r2"])

    avg_mae = np.mean(mae_scores)
    avg_mse = np.mean(mse_scores)
    avg_r2 = np.mean(r2_scores)

    return avg_mae, avg_mse, avg_r2

In [153]:
def get_optimal_hyperparameters(hp_combinations, cv_results):
    hp_combination_scores = []
    for i in range(len(hp_combinations)):
        current_hp_results = [result for result in cv_results if result["hp_combination"] == i]
        mae, mse, r2 = get_avg_scores(current_hp_results)
        hp_combination_scores.append(mse)
    optimal_combination = np.argmin(hp_combination_scores)
    optimal_hps = hp_combinations[optimal_combination]
    return optimal_hps

# Build and evaluate model

In [154]:
outer_cv_results = []

In [155]:
for current_outer_split in outer_splits:

    hp_combinations = []
    cv_results = []

    # Get training and validation sets for current outer split
    is_in_validation_set = outer_fold_ids == current_outer_split
    is_in_training_set = ~is_in_validation_set
    outer_train_features = features.loc[is_in_training_set]
    outer_train_labels = labels.loc[is_in_training_set]
    outer_val_features = features.loc[is_in_validation_set]
    outer_val_labels = labels.loc[is_in_validation_set]

    # Loop to test 10 hyperparameter combinations
    for i in range(10):
        
        # Set random hps
        kernel, criterion = get_random_hyperparameters()
        current_hps = {
            "kernel": kernel,
            "criterion": criterion
        }
        hp_combinations.append(current_hps)

        # Inner cross-validation for model selection
        for current_inner_split in inner_splits:
            print(f"\n --- Outer split {current_outer_split} - Training model {i} on inner split {current_inner_split} ---")

            # Get training and validation sets for current inner split
            is_in_validation_set = inner_fold_ids[f"inner_loop_{current_outer_split + 1}_fold_id_python"] == current_inner_split
            is_in_training_set = ~is_in_validation_set
            inner_train_features = outer_train_features.loc[is_in_training_set]
            inner_train_labels = outer_train_labels.loc[is_in_training_set]
            inner_val_features = outer_train_features.loc[is_in_validation_set]
            inner_val_labels = outer_train_labels.loc[is_in_validation_set]

            # Get model inputs
            print("Getting inputs...")
            inner_train_coords, inner_train_predictors, inner_train_target, inner_bandwidth = get_gwr_inputs(inner_train_features, inner_train_labels, bandwidth = True, kernel = kernel, criterion = criterion)
            inner_val_coords, inner_val_predictors, inner_val_target, _ = get_gwr_inputs(inner_val_features, inner_val_labels)

            # Build model
            print("Building model...")
            model = GWR(
                inner_train_coords,
                inner_train_target,
                inner_train_predictors,
                bw = inner_bandwidth,
                kernel = kernel
            )

            model.fit()

            # Get predictions
            print("Getting predictions...")
            results = model.predict(
                inner_val_coords, inner_val_predictors
            )
            predictions = results.predy

            # Get accuracy scores
            print("Evaluating predictions...")
            mae, mse, r2, moran = get_evaluation_metrics(inner_val_features, inner_val_labels, predictions)

            # Add scores for current fold to results
            cv_results.append({
                "hp_combination": i,
                "inner_split": current_inner_split,
                "hps": current_hps,
                "mae": mae,
                "mse": mse,
                "r2": r2,
                "moran": moran
            })

    print(f"\n --- Outer split {current_outer_split} - Training optimised model ---")

    # Get optimal hyperparameters for current outer split training set
    opt_hps = get_optimal_hyperparameters(hp_combinations, cv_results)

    # Get model inputs
    print("Getting inputs...")
    outer_train_coords, outer_train_predictors, outer_train_target, outer_bandwidth = get_gwr_inputs(outer_train_features, outer_train_labels, bandwidth = True, kernel = opt_hps["kernel"], criterion = opt_hps["criterion"])
    outer_val_coords, outer_val_predictors, outer_val_target, _ = get_gwr_inputs(outer_val_features, outer_val_labels)
    
    # Build model
    print("Building model...")
    model = GWR(
        outer_train_coords,
        outer_train_target,
        outer_train_predictors,
        bw = outer_bandwidth,
        kernel = opt_hps["kernel"],
    )

    model.fit()

    # Get predictions
    print("Getting predictions...")
    results = model.predict(
        outer_val_coords, outer_val_predictors
    )
    predictions = results.predy

    # Get accuracy scores
    print("Evaluating predictions...")
    mae, mse, r2, moran = get_evaluation_metrics(outer_val_features, outer_val_labels, predictions)

    outer_cv_results.append({
        "outer_split": current_outer_split,
        "hps": opt_hps,
        "mae": mae,
        "mse": mse,
        "r2": r2,
        "moran": moran,
        "inner_cv_results": cv_results
    })


 --- Outer split 0 - Training model 0 on inner split 0 ---
Getting inputs...
Building model...
Getting predictions...
Evaluating predictions...
('WARNING: ', 4, ' is an island (no neighbors)')
('WARNING: ', 5, ' is an island (no neighbors)')
('WARNING: ', 6, ' is an island (no neighbors)')
('WARNING: ', 7, ' is an island (no neighbors)')
('WARNING: ', 8, ' is an island (no neighbors)')
('WARNING: ', 9, ' is an island (no neighbors)')
('WARNING: ', 18, ' is an island (no neighbors)')
('WARNING: ', 20, ' is an island (no neighbors)')
('WARNING: ', 21, ' is an island (no neighbors)')
('WARNING: ', 35, ' is an island (no neighbors)')
('WARNING: ', 36, ' is an island (no neighbors)')
('WARNING: ', 37, ' is an island (no neighbors)')
('WARNING: ', 38, ' is an island (no neighbors)')
('WARNING: ', 39, ' is an island (no neighbors)')
('WARNING: ', 40, ' is an island (no neighbors)')
('WARNING: ', 41, ' is an island (no neighbors)')
('WARNING: ', 42, ' is an island (no neighbors)')
('WARNING: 

c:\Users\Natha\Dev\irp\.venv\Lib\site-packages\libpysal\weights\util.py:826: UserWarning: The weights matrix is not fully connected: 
 There are 561 disconnected components.
 There are 561 islands with ids: 4, 5, 6, 7, 8, 9, 18, 20, 21, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 59, 60, 61, 62, 63, 64, 65, 66, 67, 69, 84, 85, 88, 1693, 1749, 1752, 2771, 2772, 2773, 2774, 2775, 2776, 2784, 2785, 2786, 2787, 2799, 2800, 2801, 2830, 2831, 2832, 2833, 2834, 2835, 2836, 2837, 2838, 2839, 2840, 2841, 2851, 2852, 2853, 2855, 2856, 2858, 2859, 2860, 3204, 3205, 3206, 3207, 3208, 3209, 3217, 3222, 3223, 3224, 3259, 3260, 3261, 3262, 3263, 3264, 3265, 3267, 3268, 3269, 3278, 3279, 3280, 3281, 3282, 3283, 3285, 3286, 3287, 3288, 3289, 3290, 3291, 3292, 3293, 3294, 3295, 3308, 3309, 3310, 3311, 3312, 3313, 3320, 3321, 3322, 3323, 3324, 3325, 3326, 3327, 3328, 3329, 3330, 3331, 3332, 3333, 3334, 3335, 3336, 3337, 3338, 3339, 3340, 3341, 3342, 3343, 3344, 3345, 3346, 3347, 3348, 3349, 3350, 3351, 3

Building model...
Getting predictions...
Evaluating predictions...
('WARNING: ', 1336, ' is an island (no neighbors)')
('WARNING: ', 1338, ' is an island (no neighbors)')
('WARNING: ', 1339, ' is an island (no neighbors)')
('WARNING: ', 1340, ' is an island (no neighbors)')
('WARNING: ', 1341, ' is an island (no neighbors)')
('WARNING: ', 1342, ' is an island (no neighbors)')
('WARNING: ', 1343, ' is an island (no neighbors)')
('WARNING: ', 1344, ' is an island (no neighbors)')
('WARNING: ', 1345, ' is an island (no neighbors)')
('WARNING: ', 1346, ' is an island (no neighbors)')
('WARNING: ', 1347, ' is an island (no neighbors)')
('WARNING: ', 1348, ' is an island (no neighbors)')
('WARNING: ', 1349, ' is an island (no neighbors)')
('WARNING: ', 1350, ' is an island (no neighbors)')
('WARNING: ', 1351, ' is an island (no neighbors)')
('WARNING: ', 1352, ' is an island (no neighbors)')
('WARNING: ', 1353, ' is an island (no neighbors)')
('WARNING: ', 1354, ' is an island (no neighbors)

c:\Users\Natha\Dev\irp\.venv\Lib\site-packages\libpysal\weights\util.py:826: UserWarning: The weights matrix is not fully connected: 
 There are 367 disconnected components.
 There are 367 islands with ids: 1336, 1338, 1339, 1340, 1341, 1342, 1343, 1344, 1345, 1346, 1347, 1348, 1349, 1350, 1351, 1352, 1353, 1354, 1355, 1356, 1361, 1368, 1369, 1370, 1371, 1372, 1373, 1374, 1375, 1376, 1377, 1378, 1379, 1380, 1381, 1382, 1383, 1384, 1385, 1386, 1387, 1392, 1393, 1394, 1395, 1396, 1397, 1398, 1399, 1400, 1401, 1402, 1403, 1404, 1405, 1406, 1407, 1408, 1409, 1410, 1411, 1413, 1415, 1416, 1417, 1418, 1419, 1420, 1421, 1422, 1423, 1424, 1425, 1426, 1427, 1428, 1429, 1430, 1431, 1432, 1433, 1434, 1435, 1436, 1437, 1438, 1439, 1440, 1441, 1442, 1443, 1444, 1445, 1446, 1447, 1448, 1449, 1450, 1451, 1452, 1453, 1454, 1455, 1456, 1457, 1458, 1459, 1460, 1461, 1462, 1463, 1464, 1466, 1467, 1471, 1472, 1475, 1476, 1477, 1478, 1479, 1480, 1481, 1482, 1483, 1484, 1487, 1488, 1489, 1490, 1493, 1494, 1

KeyboardInterrupt: 

In [ ]:
predictor_cols = ["greenspace_proportion", "imd", "f_m_ratio", "mean_age"]
model_results = {
    "predictors": predictor_cols,
    "results": outer_cv_results
}

In [ ]:
with open("outputs/model_results/gwr.pkl", "wb") as f:
    pickle.dump(model_results, f)